In [1]:
!pip install psycopg2

You should consider upgrading via the '/Users/august/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


## Creating connection with psycopg2

In [1]:
import psycopg2
from psycopg2 import OperationalError

def create_connection(db_name, db_user, db_password, db_host, db_port):
    connection = None
    try:
        connection = psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port,
        )
        print("Connection to PostgreSQL DB successful")
    except OperationalError as e:
        print(f"The error '{e}' occurred")
    return connection

In [3]:
connection = create_connection(
    "country_app", "postgres", "", "127.0.0.1", "5432"
)

Connection to PostgreSQL DB successful


## Creating Database

In [41]:
def create_database(connection, query):
    connection.autocommit = True
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Query executed successfully")
    except OperationalError as e:
        print(f"The error '{e}' occurred")

create_database_query = "CREATE DATABASE sm_app"
create_database(connection, create_database_query)

DuplicateDatabase: database "sm_app" already exists


In [3]:
connection = create_connection(
    "sm_app", "postgres", "", "127.0.0.1", "5432"
)

Connection to PostgreSQL DB successful


In [138]:
def execute_query(connection, query):
    connection.autocommit = True
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Query executed successfully")
        return cursor
    except OperationalError as e:
        print(f"The error '{e}' occurred")

In [45]:
create_answers_table = """
    CREATE TABLE answers (
        Q TEXT,
        Theme TEXT,
        Subtheme TEXT,
        alt_op_5 TEXT,
        alt_op_4 TEXT,
        alt_op_3 TEXT,
        alt_op_2 TEXT,
        alt_op_1 TEXT,
        op_1 TEXT,
        op_2 TEXT,
        op_3 TEXT,
        op_4 TEXT,
        op_5 TEXT,
        op_6 TEXT,
        op_7 TEXT,
        op_8 TEXT,
        op_9 TEXT,
        op_10 TEXT
    )
"""

execute_query(connection,create_answers_table)

DuplicateTable: relation "answers" already exists


## Creating tables

In [12]:
create_users_table = """
CREATE TABLE IF NOT EXISTS users (
  id SERIAL PRIMARY KEY,
  name  NOT NULL, 
  age INTEGER,
  gender TEXT,
  nationality TEXT
)
"""

execute_query(connection, create_users_table)

create_posts_table = """
CREATE TABLE IF NOT EXISTS posts (
  id SERIAL PRIMARY KEY, 
  title TEXT NOT NULL, 
  description TEXT NOT NULL, 
  user_id INTEGER REFERENCES users(id)
)
"""

execute_query(connection, create_posts_table)

create_comments_table = """
    CREATE TABLE IF NOT EXISTS comments (
        post_id SERIAL PRIMARY KEY,
        text TEXT NOT NULL,
        user_id INTEGER REFERENCES users(id)
    )
"""

execute_query(connection, create_comments_table)


Query executed successfully


## Inserting Records

In [45]:
comments = [
    (1, "Cool", 2),
    (3, "That's amazing", 2),
    (5, "Tell me more", 3),
    (4, "Nice to hear", 1),
    (6, "Bullshit!", 2)
]

comments_records =  ",".join(["%s"]*len(comments))

insert_query = (
    f"INSERT INTO comments (post_id, text, user_id) VALUES {comments_records}"
)

cursor.execute(insert_query, comments)


Query executed successfully


In [16]:
users = [
    ("James", 25, "male", "USA"),
    ("Leila", 32, "female", "France"),
    ("Brigitte", 35, "female", "England"),
    ("Mike", 40, "male", "Denmark"),
    ("Elizabeth", 21, "female", "Canada"),
]

user_records = ", ".join(["%s"] * len(users))
## Generates %s, %s, %s, %s, %s

insert_query = (
    f"INSERT INTO users (name, age, gender, nationality) VALUES {user_records}"
)
## Generates INSERT INTO users (name, age, gender, nationality) VALUES %s, %s, %s, %s, %s

print(user_records)
print(insert_query)

connection.autocommit = True
cursor = connection.cursor()
cursor.execute(insert_query, users)

%s, %s, %s, %s, %s
INSERT INTO users (name, age, gender, nationality) VALUES %s, %s, %s, %s, %s


In [18]:
posts = [
    ("Happy", "I am feeling very happy today", 1),
    ("Hot Weather", "The weather is very hot today", 2),
    ("Help", "I need some help with my work", 2),
    ("Great News", "I am getting married", 1),
    ("Interesting Game", "It was a fantastic game of tennis", 5),
    ("Party", "Anyone up for a late-night party today?", 3),
]

post_records = ", ".join(["%s"] * len(posts))

insert_query = (
    f"INSERT INTO posts (title, description, user_id) VALUES {post_records}"
)

connection.autocommit = True
cursor = connection.cursor()
cursor.execute(insert_query, posts)

 ### Selecting records

In [24]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except OperationalError as e:
        print(f"The error '{e}' occurred")

select_users = "SELECT * FROM users"
users = execute_read_query(connection, select_users)

for user in users:
    print(user)

(1, 'James', 25, 'male', 'USA')
(2, 'Leila', 32, 'female', 'France')
(3, 'Brigitte', 35, 'female', 'England')
(4, 'Mike', 40, 'male', 'Denmark')
(5, 'Elizabeth', 21, 'female', 'Canada')


In [25]:
select_users_posts = """
SELECT
  users.id,
  users.name,
  posts.description
FROM
  posts
  INNER JOIN users ON users.id = posts.user_id
"""

users_posts = execute_read_query(connection, select_users_posts)

for users_post in users_posts:
    print(users_post)

(1, 'James', 'I am feeling very happy today')
(2, 'Leila', 'The weather is very hot today')
(2, 'Leila', 'I need some help with my work')
(1, 'James', 'I am getting married')
(5, 'Elizabeth', 'It was a fantastic game of tennis')
(3, 'Brigitte', 'Anyone up for a late-night party today?')


In [116]:
select_female_posts = """
SELECT 
    users.name,
    posts.description
FROM
    posts
    INNER JOIN users ON users.id = posts.user_id
WHERE
    users.gender = 'female'
"""

select_all = """
SELECT * FROM posts
"""

cursor.execute(select_all)
results = cursor.fetchall()

for result in results:
    print(result)

(1, 'Happy', 'I am feeling very happy today', 1)
(2, 'Hot Weather', 'The weather is very hot today', 2)
(3, 'Help', 'I need some help with my work', 2)
(4, 'Great News', 'I am getting married', 1)
(5, 'Interesting Game', 'It was a fantastic game of tennis', 5)
(6, 'Party', 'Anyone up for a late-night party today?', 3)


In [74]:
select_posts_with_comment = """
SELECT
    posts.description as post,
    comments.text as comment,
    users.name
FROM
    posts
    INNER JOIN comments ON posts.id = comments.post_id 
    INNER JOIN users ON users.id = comments.user_id
WHERE 
    users.name IN ('Leila', 'Brigitte')
ORDER BY 
    users.name
"""

cursor = connection.cursor()
cursor.execute(select_posts_with_comment)
results = cursor.fetchall()

for result in results: 
    print(result)


('It was a fantastic game of tennis', 'Tell me more', 'Brigitte')
('I am feeling very happy today', 'Cool', 'Leila')
('I need some help with my work', "That's amazing", 'Leila')
('Anyone up for a late-night party today?', 'Bullshit!', 'Leila')


### Showing columns of selection

In [12]:
cursor = connection.cursor()
cursor.execute(select_posts_with_comment)
cursor.fetchall()

column_names = [description[0] for description in cursor.description]
print(column_names)

NameError: name 'select_posts_with_comment' is not defined

## Updating Table Records

In [145]:
update_post_description = """
UPDATE
  posts
SET
  description = 'The weather has become better now'
WHERE
  id = 2
"""
execute_query(connection,  update_post_description)

select_post_description = """
SELECT * FROM
  posts
WHERE
  id = 2
"""

cursor = connection.cursor()
cursor.execute(select_post_description)
results = cursor.fetchall()

for r in results:
  print(r)

Query executed successfully
(2, 'Hot Weather', 'The weather has become better now', 2)


## Deleting Table Records

In [160]:
## Delete record
delete_comment = "DELETE FROM comments WHERE post_id = 5"
execute_query(connection, delete_comment)

select_post_description = """
SELECT * FROM
  comments
  ORDER BY post_id
"""

cursor = connection.cursor()
cursor.execute(select_post_description)
results = cursor.fetchall()

for r in results:
  print(r)

Query executed successfully
(1, 'Cool', 2)
(3, "That's amazing", 2)
(4, 'Nice to hear', 1)
(6, 'Bullshit!', 2)


In [161]:
# Restor removed record
insert_query = (
    f"INSERT INTO comments (post_id, text, user_id) VALUES (5,'What?',3)"
)

connection.autocommit = True
cursor = connection.cursor()
cursor.execute(insert_query)

cursor = connection.cursor()
cursor.execute(select_post_description)
results = cursor.fetchall()

for r in results:
  print(r)

(1, 'Cool', 2)
(3, "That's amazing", 2)
(4, 'Nice to hear', 1)
(5, 'What?', 3)
(6, 'Bullshit!', 2)


## Loading CSV

### Pick out headers from file to input to SQL for table creation

In [18]:
cursor = connection.cursor()
with open('./data/WVS_Cross-National_Wave_7_csv_v2_0.csv', 'r') as f:
    # Notice that we don't need the `csv` module.
    headers = next(f) # Skip the header row.

head = headers.replace('\ufeff','').strip().replace(';',',').split(',')

new_headers = ''
for i in range(len(head)):
    head[i] = head[i] + ' TEXT'
    new_headers += head[i]+','


new_headers = new_headers[:-1]

print(new_headers)

create_answers_table = f"""
    CREATE TABLE questions_large (
        {new_headers}
    )
"""
with connection:
        with connection.cursor() as cursor:
            cursor.execute(create_answers_table)
            results = cursor.fetchall()

print(results)            
#print(create_answers_table)
#cursor.execute(create_answers_table)


"version" TEXT,"doi" TEXT,"A_WAVE" TEXT,"A_STUDY" TEXT,"B_COUNTRY" TEXT,"B_COUNTRY_ALPHA" TEXT,"C_COW_NUM" TEXT,"C_COW_ALPHA" TEXT,"A_YEAR" TEXT,"D_INTERVIEW" TEXT,"J_INTDATE" TEXT,"FW_END" TEXT,"FW_START" TEXT,"K_TIME_START" TEXT,"K_TIME_END" TEXT,"K_DURATION" TEXT,"Q_MODE" TEXT,"N_REGION_ISO" TEXT,"N_REGION_WVS" TEXT,"N_TOWN" TEXT,"G_TOWNSIZE" TEXT,"G_TOWNSIZE2" TEXT,"H_SETTLEMENT" TEXT,"H_URBRURAL" TEXT,"I_PSU" TEXT,"O1_LONGITUDE" TEXT,"O2_LATITUDE" TEXT,"S_INTLANGUAGE" TEXT,"LNGE_ISO" TEXT,"E_RESPINT" TEXT,"F_INTPRIVACY" TEXT,"E1_LITERACY" TEXT,"W_WEIGHT" TEXT,"S018" TEXT,"PWGHT" TEXT,"S025" TEXT,"Q1" TEXT,"Q2" TEXT,"Q3" TEXT,"Q4" TEXT,"Q5" TEXT,"Q6" TEXT,"Q7" TEXT,"Q8" TEXT,"Q9" TEXT,"Q10" TEXT,"Q11" TEXT,"Q12" TEXT,"Q13" TEXT,"Q14" TEXT,"Q15" TEXT,"Q16" TEXT,"Q17" TEXT,"Q18" TEXT,"Q19" TEXT,"Q20" TEXT,"Q21" TEXT,"Q22" TEXT,"Q23" TEXT,"Q24" TEXT,"Q25" TEXT,"Q26" TEXT,"Q27" TEXT,"Q28" TEXT,"Q29" TEXT,"Q30" TEXT,"Q31" TEXT,"Q32" TEXT,"Q33" TEXT,"Q33_3" TEXT,"Q34" TEXT,"Q34_3" TEXT,"

DuplicateTable: relation "questions_large" already exists


In [16]:
connection = create_connection(
    "country_app", "postgres", "", "127.0.0.1", "5432"
)

Connection to PostgreSQL DB successful


In [7]:
import os 
os.getcwd()

'/Users/august/Documents/Programming/country_viz'

### Pick out headers as iterable to input to cursor.copy_from

In [95]:
cursor = connection.cursor()
with open('./data/WVS_Cross-National_Wave_7_csv_v2_0.csv', 'r') as f:
    # Notice that we don't need the `csv` module.
    headers = next(f) # Skip the header row.

head = headers.replace('\ufeff','').strip().replace(';',',').split(',')
head=list(map(lambda a:a[1:-1],head))

### Loading answers and questions files

In [48]:
## Load answers file
cursor = connection.cursor()
with open('./data/answers_v2.csv', 'r') as f:
    # Notice that we don't need the `csv` module.
    header = next(f) # Skip the header row.
    cursor.copy_from(f, 'answers_2', sep=';')
    
connection.commit()

In [94]:
## Load questions file, columns make sure not all columns are included
cursor = connection.cursor()
with open('./data/WVS_Cross-National_Wave_7_cleaning_2.csv', 'r') as f:
    # Notice that we don't need the `csv` module.
    next(f) # Skip the header row.
    
    # head[:]
    cursor.copy_from(f, 'questions', sep=';', columns=head[:95])

connection.commit()

# Grouping large questions

In [1]:
import pandas as pd

answer_dict = {'-5':'alt_op_5','-4':'alt_op_4','-3':'alt_op_3','-2':'alt_op_2','-1':'alt_op_1','1':'op_1','2':'op_2','3':'op_3','4':'op_4','5':'op_5','6':'op_6','7':'op_7','8':'op_8','9':'op_9','10':'op_10'}

def ask_question(question_nr, country_alpha):
    select_question_answers = f"""
        SELECT "Q{question_nr}", COUNT("Q{question_nr}") FROM QUESTIONS WHERE "B_COUNTRY_ALPHA" = '{country_alpha}' GROUP BY "Q{question_nr}" ORDER BY "Q{question_nr}"
    """

    cursor = connection.cursor()
    cursor.execute(select_question_answers)
    results = cursor.fetchall()

    cols = ''
    freq = []
    for r in results:
        cols += (answer_dict[str(r[0])])+','
        freq.append(r[1])

    ## Calculate share
    sum = 0
    for opt in freq:
        sum += int(opt)
    freq=list(map(lambda a:round((a/sum),4)*100,freq))


    select_answers = f"""SELECT q,theme,subtheme,{cols[:-1]} FROM ANSWERS_2 WHERE "q" = '{question_nr}.0'"""
    select_columns = """SELECT column_name, ordinal_position FROM information_schema.columns WHERE table_name = 'answers_2'"""

    def get_selection(query): 
        cursor = connection.cursor()
        cursor.execute(query)
        results = cursor.fetchall()

        return results
            
    question_details = get_selection(select_answers)[0]
    freq = list(map(lambda a:str(round(a,4))+"%",freq)) ## add percentages, but string
    data = [*list(question_details[0:3]),*freq,sum]
    
    columns = [*['Question','Theme','Subtheme'],*question_details[3:],*['Answers']]
    df = pd.DataFrame([data],columns=columns)
    print(country_alpha)
    display(df)
    return df
    

question_nr = 49
ask_question(question_nr, 'AND')
ask_question(question_nr, 'IRQ')
ask_question(question_nr, 'JPN')


NameError: name 'connection' is not defined

TypeError: '<' not supported between instances of 'int' and 'str'

In [209]:
import pandas as pd


,question,theme,subtheme,Don't know,No answer,Very important,Rather important,Not very important,Not at all important
0,4.0,Important in life,Politics,25,5,211,257,333,369


In [237]:
columns = [*['question','theme','subtheme'],*question_details[3:]]
columns

['question',
 'theme',
 'subtheme',
 "Don't know",
 'No answer',
 'Very important',
 'Rather important',
 'Not very important',
 'Not at all important']

['4.0', 'Important in life', 'Politics', 25, 5, 211, 257, 333, 369]

In [231]:
freq

[25, 5, 211, 257, 333, 369]